In [1]:
import math
import random
import numpy
from collections import Counter
import json
import pickle

# algunas funciones!

In [2]:
def distribucion(P):
    D=[]
    for i in range(len(P)):
        D+=[distribucionk(P,i)]
    return [sum(k)/len(P) for k in zip(*D)]

def distribucionk(P,k):##numero de significados para cada palabra del bicho k
    d=[]
    for i in range(n):
        d+=[sum(P[k][i,:])]
   
    return d

def EL(P): ##vocabulario efectivo
    a=0
    for k in range(p):
        for i in range(n):
            if sum(P[k][i,:])>0:
                a+=1

    return a/float(p*n)

def acuerdo(P): ##distancia a matriz promedio
    b=numpy.zeros((n,m))
    c=0

    for k in range(p):
        b+=P[k]
    b=1/float(p)*b
    for k in range(p):
        c+=numpy.sum(b!=P[k])

    return c/float(p*n*m)

# simulacion!

In [ ]:
p=100
n=64
m=64
copias=10
time=200000
D=[]
V=[]
A=[]
Pop=[]

THETA=[]
for k in range(51):
    THETA+=[k/float(50)]
    
for theta in THETA:
    
    print(theta)
    dd=[] ##distribución
    v=[] ##vocabulario
    a=[] ##acuerdo
    
    for c in range(copias):
        
        P=[0]*p
        for i in range(p):
            P[i]=numpy.zeros((n,m))
        for i in range(p):
            for r in range(n):
                for s in range(m):
                    if random.random()<0.5: ##llenamos con parámetro 0.5
                        P[i][r][s]=1    
                        
        for t in range(time):
                        
            S=random.randint(0,p-1) ##speaker
            H=random.randint(0,p-1) ##hearer          
            #H=random.choice(S-1,S+1)  ## agregar r!
            #ahora viene lo bueno! S elige un tópico como el significado que será discutido en la 
            #interacción

            M=random.randint(0,m-1)

            #con esta información (compartida con H al menos en principio), S prefiere -porque es muy flojo- la palabra que asociada
            #a M tenga el mayor número de conexiones. O sea, podemos definir una probabilidad 
            #si M está wacho, N es al azar. Sino, se pone interesante
            #recorremos la columna M de la matriz lexica de S y buscamos el conjunto de las palabras conectadas, C

            C=[] 
            G=[] #aquí guardamos el grado de las palabras de C
            Proba=[] #la probabilidad

            for i in range(n):
                if P[S][i][M]==1:
                    C+=[i]
      
            if len(C)==0: #wacho
               N=random.randint(0,m-1)
   
            else: #contamos el número de significados conectados con las palabras de C
                for item in C:
                    G+=[sum(P[S][item])] #sumamos por fila

                #ahora en Proba repetimos los elementos de C de acuerdo con la información de G
                for i in range(len(C)):
                    #Proba+=[C[i]]*(G[i])
                    Proba+=[C[i]]*int((G[i]))

                #S elige una palabra de Proba ###la más conectada
        
                if random.random()>theta:
                    N1=[item for item in Counter(Proba).most_common()]
                    N=N1[len(N1)-1][0]## menos conectado
                else:
                    N=[item for item in Counter(Proba).most_common()][0][0]## mas conectado

   #H recibe la palabra N

   #naming game!
    
            if P[H][N][M]==0:
                P[H][N][M]=1
            else:
      
                for i in range(n):
                    P[H][i][M]=0
                    P[S][i][M]=0
                P[H][N][M]=1
                P[S][N][M]=1
                
          
        C=distribucion(P)
        AA=acuerdo(P)
        B=EL(P)  
        dd+=[C]
        v+=[B]
        a+=[AA]
        
    D+=[dd]
    V+=[v]
    A+=[a]
    Pop+=[P]
    
pickle.dump(D,open('D.p','wb')) ##distribucion
pickle.dump(V,open('V.p','wb')) ##vocabulario
pickle.dump(A,open('A.p','wb')) ##acuerdo
pickle.dump(Pop,open('P.p','wb')) ##poblacion (matrices!!!)

0.0
0.02
0.04
0.06
0.08
0.1
0.12
0.14
0.16
0.18
0.2
0.22
0.24
0.26
0.28
0.3
0.32
0.34
0.36
0.38
0.4
0.42
0.44
0.46
0.48
0.5
0.52
0.54
0.56
0.58
0.6
0.62
0.64
0.66
0.68
0.7
0.72
0.74
0.76
0.78
0.8
0.82
0.84
0.86
0.88
0.9
0.92
0.94
0.96
0.98
1.0
